In [2]:
import math
from pyplasm import *
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.006592 seconds


/usr/local/lib/python2.7/dist-packages/larlib/larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


In [3]:
PARAMS = {
    'common':
    {
        'circle_vertices': 60,
        'curve_radius': 0.02
    },
    'colors':
    {
        'wood': (191, 161, 93),
        'metal': (110, 110, 110)
    },
    'desk':
    {
        'x': 0.7,
        'y': 0.5,
        'z': 0.8,
        'plane_width': 0.02,
        'ud_percent': 0.1,
        'ud_width': 0.01,
        'legs_percent': 0.95,
        'scale_factor1': 0.88,
        'scale_factor2': 0.77,
        'scale_factor3': 0.8
    },
    'leg':
    {
        'radius': 0.01,
        'foot_z': 0.005,
        'foot_r_percent': 1.1
    },
    'chair':
    {
        'x': 0.4,
        'y': 0.4,
        'z': 0.8
    },
    'bookshelf':
    {
        'shelf_z': 0.6,
        'bookshelf_width': 0.02
    }
}

In [7]:
# wrapper to color with a 0..255 scale
rgba = lambda r, g, b, a=1: COLOR([r / 255.0, g / 255.0, b / 255.0, a])

def circle(origin, radius, start = 0, end = (2 * pi)):
    """
        Returns a list of vertex describing the rotation
        between start and end angles
        Args:
            origin: the origin of rotation
            radius: radius of rotation
            start: start angle
            end: end angle
    """
    v = int(PARAMS['common']['circle_vertices'] * (end - start) / (2 * pi))
    inc = (2 * pi) / float(PARAMS['common']['circle_vertices'])
    get_angle = lambda w: start + (inc * w)
    x, y, z = origin
    circle = [[x + (cos(get_angle(i)) * radius), y + (sin(get_angle(i)) * radius), z]
           for i in range(0, v)]
    return circle


def rounded_corner(origin, radius, q):
    """
        Returns a list of vertex describing a rounded corner
        Args:
            origin: the origin of the rotation
            radius: radius of the rotation
            q: quadrant of rotation
    """
    start_angle = (0, (pi / 2.0), pi, ((3 * pi) / 2.0))[q - 1]
    return circle(origin, radius, start_angle, start_angle + (pi / 2.0))


def rounded_rectangle(x, y, radius):
    """
        Returns an two-dimensional HPC with rounded corners
        Args:
            x: area width
            y: area height
            radius: curve of corner
    """
    angle1 = rounded_corner((x - radius, y - radius, 0), radius, 1)
    angle2 = rounded_corner((radius,  y - radius, 0), radius, 2)
    angle3 = rounded_corner((radius, radius, 0), radius, 3)
    angle4 = rounded_corner((x - radius, radius, 0), radius, 4)

    plane = [angle1 + angle2 + angle3 + angle4,
             [[i for i in range(1, (len(angle1) * 4) + 1)]],
             None]
    return MKPOL(plane)


def legs(offset, size):
    """
        Returns an HPC describing the four legs of a table or a chair
        Args:
            offset: distance between the sides of the area
            size: size of the area to which the legs will be attached
    """
    legsParams = PARAMS['leg']
    x, y, z = size
    z -= legsParams['foot_z']

    # make pols
    legPol = CYLINDER([legsParams['radius'], z])(PARAMS['common']['circle_vertices'])

    legFootPol = CYLINDER([legsParams['radius'] * legsParams['foot_r_percent'],
                           legsParams['foot_z']])(PARAMS['common']['circle_vertices'])

    # color
    legPol = rgba(*PARAMS['colors']['metal'])(legPol)
    legFootPol = rgba(0, 0, 0)(legFootPol)

    # make final leg
    legPol = STRUCT([legFootPol,
                     T(3)(legsParams['foot_z']),
                     legPol])

    # make legs
    legsPol = STRUCT([T([1,2])([offset, offset]),
                      legPol,
                      T([1,2])([x - (offset * 2), 0]),
                      legPol,
                      T([1,2])([0, y - (offset * 2)]),
                      legPol,
                      T([1,2])([-(x - (offset * 2)), 0]),
                      legPol])

    return legsPol

def ggpl_simple_chair():
    """
        Returns an HPC describing a chair (unfinished)
    """

    return STRUCT([legs(0.02, (0.4, 0.4, 0.6)), T(3)(0.6),
                 OFFSET([0,0, 0.02])(rounded_rectangle(0.4, 0.4, 0.02))])


def ggpl_desk():
    """
        Returns an HPC describing a desk
    """
    deskParams = PARAMS['desk']

    scale1 = lambda x: x * deskParams['scale_factor1']
    scale2 = lambda x: x * deskParams['scale_factor2']

    offsetX = lambda f: (deskParams['x'] - f(deskParams['x'])) / 2.0
    offsetY = lambda f: (deskParams['y'] - f(deskParams['y'])) * deskParams['scale_factor3']

    offsetX1 = offsetX(scale1)
    offsetX2 = offsetX(scale2)
    offsetY1 = offsetY(scale1)
    offsetY2 = offsetY(scale2)


    underDeskHeight = deskParams['z'] * deskParams['ud_percent']


    # make pols
    legsPol = legs(PARAMS['common']['curve_radius'],
                   (deskParams['x'], deskParams['y'],
                    deskParams['z'] - deskParams['plane_width']))


    underDeskPol = MKPOL([
        [
            [
                offsetX1,
                offsetY1,
                underDeskHeight
            ],
            [
                offsetX1,
                offsetY1 + scale1(deskParams['y']),
                underDeskHeight
            ],
            [
                offsetX1 + scale1(deskParams['x']),
                offsetY1,
                underDeskHeight
            ],
            [
                offsetX1 + scale1(deskParams['x']),
                offsetY1 + scale1(deskParams['y']),
                underDeskHeight
            ],
            [
                offsetX2,
                offsetY2,
                0
            ],
            [
                offsetX2,
                offsetY2 + scale2(deskParams['y']),
                0
            ],
            [
                offsetX2 + scale2(deskParams['x']),
                offsetY2,
                0
            ],
            [
                offsetX2 + scale2(deskParams['x']),
                offsetY2 + scale2(deskParams['y']),
                0
            ]
        ],
        [
            [5, 6, 7, 8],
            [1, 2, 5, 6],
            [2, 4, 6, 8],
            [3, 4, 7, 8]
        ],
        None
    ])

    planePol = rounded_rectangle(deskParams['x'],
                                 deskParams['y'],
                                 PARAMS['common']['curve_radius'])

    # make offsets
    underDeskPol = OFFSET([deskParams['ud_width'],
                           deskParams['ud_width'],
                           deskParams['ud_width']])(underDeskPol)

    planePol = OFFSET([0, 0, deskParams['plane_width']])(planePol)

    # colorize
    underDeskPol = rgba(*PARAMS['colors']['metal'])(underDeskPol)
    planePol = rgba(*PARAMS['colors']['wood'])(planePol)

    # create final HPC
    deskPol = STRUCT([underDeskPol, T(3)(underDeskHeight), planePol])
    deskPol = STRUCT([legsPol,
                      T(3)((deskParams['z'] -
                            deskParams['plane_width']) -
                           underDeskHeight),
                      deskPol])

    return deskPol


def ggpl_bookshelf(x, y, shelves):
    """
        Returns an HPC object describing a bookshelf
        Args:
            x: size (x-axis)
            y: size (y-axis)
            shelves: number of shelves
    """
    bookshelfParams = PARAMS['bookshelf']
    v = []
    f = []

    baseShelf = [(0, 0, 0), (x, 0, 0), (0, y, 0), (x, y, 0)]

    # utility functions

    # addZ([x, y, w], z) -> [x, y, w + z]
    addZ = lambda a, z: [a[0], a[1], a[2] + z]
    addShelf = lambda z: [addZ(baseShelf[0], z), addZ(baseShelf[1], z),
                          addZ(baseShelf[2], z), addZ(baseShelf[3], z)]

    for i in range(0, shelves + 1):
        point = i * 4
        v += addShelf(bookshelfParams['shelf_z'] * i)
        f.append([(point + 1), (point + 2), (point + 3), (point + 4)])

    last = len(v)
    f += [[3, 4, last - 1, last],
          [1, 3, last - 3, last - 1],
          [2, 4, last - 2, last]]

    # make pol

    bookshelfPol = MKPOL([v, f, None])

    # make offset

    bookshelfPol = OFFSET([bookshelfParams['bookshelf_width'],
                           bookshelfParams['bookshelf_width'],
                           bookshelfParams['bookshelf_width']])(bookshelfPol)

    # color

    bookshelfPol = rgba(*PARAMS['colors']['wood'])(bookshelfPol)

    return bookshelfPol

#### Simple Chair (unfinished)

In [8]:
VIEW(ggpl_simple_chair())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f7a18575030> >

![](chair.png)

#### Desk

In [11]:
VIEW(ggpl_desk())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f79e41b4450> >

![](desk.png)

#### Bookshelf

In [14]:
VIEW(ggpl_bookshelf(2, 0.6, 5))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f79e41f9030> >

![](bookshelf.png)